In [24]:
import requests
import pandas as pd
from IPython.display import Image
from IPython.core.display import HTML 

In [1]:
def get_authentication():
    """
    Get authentication from Spotify API using the credentials we have 
    """
    AUTH_URL = 'https://accounts.spotify.com/api/token'
    cid ="0753888cfb694cf98b9332cd343859a3" 
    secret = "0f47a99eb4bc40eea4f3c9d6b3f1df82"
    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': cid,
        'client_secret': secret,
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()
    # save the access token
    access_token = auth_response_data['access_token']

    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)}
    return headers 


In [95]:
def get_artist_id(artist: str):
    """
    Use the searching function in Spotify API to search for the most relavant artist name
    Input: String of artist name
    Output: String of artist id 
    """
    URI = 'https://api.spotify.com/v1/search?query='+ artist +'&offset=0&limit=1&type=artist'
    response = requests.get(url = URI, headers=get_authentication())
    response = response.json()
    artist_id = response['artists']['items'][0]['id']
    return artist_id

In [91]:
def get_artist_album(artist_id: str, limit: int):
    """
    Get the album from the artist, you can specify the number of albums returned
    Input: String of artist id, Number of albums
    Output: Dataframe with relavant album information
    """
    album_url = 'https://api.spotify.com/v1/artists/' + artist_id + '/albums?limit=' + str(limit) + '&include_groups=album'
    response = requests.get(album_url, headers=get_authentication())
    response = response.json()
    df = pd.DataFrame(response['items']) # we make the data frame using the items 
    df['image_url'] = df['images'].apply(lambda x: x[0]['url']) # we need only one image url
    df['available_market_cnt'] = df['available_markets'].apply(lambda x: len(x))
    df['release_date'] = pd.to_datetime(t['release_date'] ) # change the string to datetime
    df.sort_values(by='release_date', ascending=False) # descending release time
    df = df.loc[:, ['name','id', 'release_date', 'total_tracks','image_url','available_market_cnt' ]] 
    return df 

In [69]:
# functions for printing out pretty images 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'


def output_dataframe(df):
    pd.set_option('display.max_colwidth', None)
    image_cols = ['image_url']  #<- define which columns will be used to convert to html
# Create the dictionariy to be passed as formatters
    format_dict = {}
    for image_col in image_cols:
        format_dict[image_col] = path_to_image_html
    display(HTML(df.to_html(escape=False ,formatters=format_dict)))


In [93]:
artistName ='Illenium'
artist_id = get_artist_id(artistName)
t = get_artist_album(artist_id, 50)
output_dataframe(t)

,name,id,release_date,total_tracks,image_url,available_market_cnt
0,Fallen Embers (Deluxe Version),0KgcBsDY5uFt4EaG3roI3x,2021-10-22,19,,184
1,Fallen Embers,4YomxhmDglPvD93W3T6l1V,2021-07-16,14,,184
2,ASCEND (Remixes),2Ds60s4LcHcb77dNQm22kA,2020-05-15,24,,184
3,ASCEND,60xcVwuQJAOyu11xf9mObS,2019-08-16,17,,182
4,Awake (Remixes),4f9voI47wfAZIp0UgwGy6o,2018-06-18,15,,184
5,Awake,5rvIgXvAPGWspXY4rDLkeU,2017-09-21,13,,184
6,Ashes (Remixes),2R7PeUrbSseLkirdasHPqg,2016-12-16,15,,184
7,Ashes,1hG9qANsJg5lYm36MYun97,2016-02-15,12,,184


In [116]:
def get_album_tracks(df):
    """
    For all the albums of a artist, get the list of track id 
    Input: dataframe 
    Output: list of tracks 
    """
    track_id_list = []
    for i in df['id']:
        url =  'https://api.spotify.com/v1/albums/' + i + '/tracks?limit=50' 
        response = requests.get(url = url, headers=get_authentication())
        response = response.json()
        track_id = [response['items'][i]['id'] for i in range(len(response['items']))]
        track_id_list.append(track_id)
    return track_id_list

In [118]:
album_list = get_album_tracks(t)

In [119]:
album_list[0]

['2Ms30hRxAxBpoZmh6WZvoo',
 '72srz1PlxoUUNiw6JcqaaI',
 '0jitsmziKMI9MpzENnB0Tc',
 '1dVn0YP06AMySJXG6hdC55',
 '5zRljvpgY1elGsc18Qp3tC',
 '4eW9LBbyJcSeJ3wp9Q83s1',
 '3ABiNXqIEkDx8jrcMpSTQt',
 '6CFm4rn5abOUJhONplxMjj',
 '6XYSeb52faaNtl53CHIAVj',
 '2G6MoKYA9jzrbscPiwsbdg',
 '5r7Lq0f8mFem1O0p7V07T4',
 '1hKP9uIcZe0VyXQ2fdRQbq',
 '5B5932ySoLMVaOuqXdDgkW',
 '1r438dOUJRtUSFv54T97Fp',
 '2pKeerhKOKCx3Fid0GhikL',
 '7Big35moOAF5WtTgkUN0pq',
 '4LyisbIqxl0KkBkurRPjzJ',
 '7bV4ng1yiKLAvryrdTxBYO',
 '4FKWBbeeKrX4iDOnkEklfv']

In [129]:
# I need track popularity (can't get the play counts )
# and track audio features 
url =  'https://api.spotify.com/v1/tracks/' +'4eW9LBbyJcSeJ3wp9Q83s1' +'/' 
response = requests.get(url = url, headers=get_authentication())


In [145]:
def get_popularity(album_track_list):
    track_dict = {'id':[], 'name':[], 'duration':[], 'popularity':[]}
    for album in album_track_list:
        for track in album:
            url =  'https://api.spotify.com/v1/tracks/' + track +'/' 
            response = requests.get(url = url, headers=get_authentication())
            a = response.json()
            track_dict['id'].append(track)
            track_dict['name'].append(a['name'])
            track_dict['duration'].append(a['duration_ms']/1000)
            track_dict['popularity'].append(a['popularity'])
    return pd.DataFrame.from_dict(track_dict)

In [139]:
d = {'id':['4eW9LBbyJcSeJ3wp9Q83s1'], 'name': [ a['name']], 'duration':[ a['duration_ms']/1000], 'popularity':[a['popularity'] ]}

In [146]:
get_popularity(album_list)

,id,name,duration,popularity
0,2Ms30hRxAxBpoZmh6WZvoo,Wouldn’t Change A Thing,187.000,55
1,72srz1PlxoUUNiw6JcqaaI,Blame Myself - ILLENIUM and Virtual Riot Remix,198.857,51
2,0jitsmziKMI9MpzENnB0Tc,Hurts Like This,238.095,59
3,1dVn0YP06AMySJXG6hdC55,Superhero,227.000,53
4,5zRljvpgY1elGsc18Qp3tC,I See You,345.436,51
...,...,...,...,...
124,60NXuYborLfMiTPJMcxkM5,Only One,252.000,49
125,1uLO8rvBwJoRqrLIyvQNWF,I'll Be Your Reason,219.882,37
126,3MtxiqrTNRwtj9EIVDjRSU,Afterlife,364.090,51
127,33eO3yOyKqrOSySHy0Bo8T,Jester - Bonus Track,181.767,0
